In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import os 
from loaddata import MultiModalLoader
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from model import MultistainModel
import torch.optim as optim
from tqdm import tqdm
from torchmetrics import AUROC
from torch.utils.tensorboard import SummaryWriter




# Load Data

In [2]:
f = "data/datatable.csv"
filepath= "results"

n_mods = 3
n_epochs = 2

train_DS = MultiModalLoader(f, "TRAIN",n_mods=n_mods)
test_DS = MultiModalLoader(f, "TEST",n_mods=n_mods)
valid_DS = MultiModalLoader(f, "VALIDATION",n_mods=n_mods)

In [3]:
train_loader = DataLoader(train_DS, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_DS, batch_size=32, shuffle=False)
test_loader = DataLoader(test_DS, batch_size=32, shuffle=False)



# Create Model

In [4]:
writer = SummaryWriter()

model = MultistainModel(n_classes = 2)
#model = torch.compile(premodel)  # TODO not working yet 
criterion = nn.CrossEntropyLoss()  
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # TODO optimal optimizer for this task?
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.9)  # TODO optimal scheduler? 
auroc = AUROC(task="multilabel", num_labels=2)  # TODO do for categorical
activate = nn.Softmax(dim=1)  # TODO for catefgorical needs to be a softmax



# Network Graph

In [5]:
inputs = torch.randn(1,3,3,224, 224)

# Define the SummaryWriter object
writer = SummaryWriter()

# Add the graph of the model to TensorBoard
writer.add_graph(model, inputs)

# Training Loop

In [6]:

for epoch in range(n_epochs):
    model.train()
    train_loss = []
    train_auroc = []
    with tqdm(train_loader, unit="batch") as tepoch:
        for idx,(x, y) in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch+1}|Train")

            pred = model(x)
            loss = criterion(pred,y.float())
            loss.backward()
            optimizer.step()
            
            tr_auroc = auroc(activate(pred), y.int())

            train_loss.append(loss.item())
            train_auroc.append(tr_auroc)
            tepoch.set_postfix(train_loss=loss.item(), train_AUROC=tr_auroc.item())

            writer.add_scalar('training_loss', loss.item(), global_step=epoch * len(train_loader) + idx)
            writer.add_scalar('training_auroc', tr_auroc.item(), global_step=epoch * len(train_loader) + idx)

            for name, param in model.named_parameters():
                writer.add_histogram(name, param, global_step=epoch * len(train_loader) + idx)
            if idx>0:  # get rid of before running 
                break
    mean_train_loss = np.mean(train_loss)  
    mean_train_auroc = np.mean(train_auroc) 

    scheduler.step()
    valid_loss = []
    valid_auroc = []
    model.eval()
    with tqdm(valid_loader, unit="batch") as vepoch:
        for idx,(x, y) in enumerate(vepoch):
            vepoch.set_description(f"Epoch {epoch+1}|Valid")
            out = model(x)
            
            loss= criterion(out,y.float())
            val_auroc  = auroc(activate(out), y.int())
            
            valid_loss.append(loss.item())
            valid_auroc.append(val_auroc)
            vepoch.set_postfix(valid_loss=loss.item(), valid_AUROC=val_auroc.item())

            writer.add_scalar('validation_loss', loss.item(), global_step=idx)
            writer.add_scalar('validation_auroc', val_auroc.item(), global_step=idx)


            if idx>0: # TODO get rid of before running 
                break

    mean_valid_loss = np.mean(valid_loss)  
    mean_valid_auroc = np.mean(valid_auroc) 


writer.close()

state = {
    'epoch': epoch,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    }
torch.save(state, filepath+"/model.pt")

Epoch 1|Train:   2%|▏         | 1/44 [01:11<50:56, 71.08s/batch, train_AUROC=0.477, train_loss=0.735]
Epoch 1|Valid:  14%|█▍        | 1/7 [00:30<03:05, 30.96s/batch, valid_AUROC=0.301, valid_loss=0.703]
Epoch 2|Train:   2%|▏         | 1/44 [00:59<39:22, 54.95s/batch, train_AUROC=0.627, train_loss=0.704]

## Evaluate

In [ ]:
test_loss = []
test_aurocs = []
model.eval()
results = []
with tqdm(test_loader, unit="batch") as testepoch:
    for idx,(x, y) in enumerate(testepoch):
        testepoch.set_description(f"Test")
        out = model(x)
        
        loss = criterion(out,y.float())
        test_auroc = auroc(activate(out), y.int())

        results.append( torch.cat((y.float(),activate(out)),dim=1) )
        
        test_loss.append(loss.item())
        test_aurocs.append(test_auroc.item())
        testepoch.set_postfix(test_loss=loss.item(), test_AUROC=val_auroc.item())


        if idx>0: # TODO get rid of before running 
            break

    mean_test_loss = np.mean(test_loss)  # TODO save in tensorboard
    mean_test_auroc = np.mean(test_aurocs)  # TODO save in tensorboard
    results_df = pd.DataFrame(torch.cat(results).detach().numpy(),columns=["cls1_label","cls2_label","cls1_label","cls2_pred"])  # hardcoded for 2 classes only

In [ ]:

os.path.exists(filepath)
if not os.path.exists(filepath):
    os.mkdir(filepath)
f = results_df.to_csv((filepath+"/result_table.csv"))
#  TODO settings store as file

In [ ]:
import torch
device =torch.device("cpu")

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Devicetype {device.type}")